In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
import seaborn as sns

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

In [3]:
cal=["LotShape","HouseStyle","MasVnrType","ExterQual","Foundation","BsmtQual","BsmtFinType1","HeatingQC","KitchenQual","FireplaceQu","GarageFinish"]

In [4]:
x_train=df_train.loc[:,cal]
x_test=df_test.loc[:,cal]

In [5]:
x_test.isnull().sum()

LotShape          0
HouseStyle        0
MasVnrType       16
ExterQual         0
Foundation        0
BsmtQual         44
BsmtFinType1     42
HeatingQC         0
KitchenQual       1
FireplaceQu     730
GarageFinish     78
dtype: int64

In [6]:
x_train.loc[x_train.isnull()["GarageFinish"],:] = x_train.loc[x_train.isnull()["GarageFinish"],:].fillna(value="None")
x_train.loc[x_train.isnull()["FireplaceQu"],:] = x_train.loc[x_train.isnull()["FireplaceQu"],:].fillna(value="None")
x_train.loc[x_train.isnull()["BsmtQual"],:] = x_train.loc[x_train.isnull()["BsmtQual"],:].fillna(value="None")
x_train.loc[x_train.isnull()["BsmtFinType1"],:] = x_train.loc[x_train.isnull()["BsmtFinType1"],:].fillna(value="None")
x_train.loc[x_train.isnull()["MasVnrType"],:] = x_train.loc[x_train.isnull()["MasVnrType"],:].fillna(value="None")

In [7]:
x_test.loc[x_test.isnull()["KitchenQual"],:] = x_test.loc[x_test.isnull()["KitchenQual"],:].fillna(value="None")
x_test.loc[x_test.isnull()["GarageFinish"],:] = x_test.loc[x_test.isnull()["GarageFinish"],:].fillna(value="None")
x_test.loc[x_test.isnull()["FireplaceQu"],:] = x_test.loc[x_test.isnull()["FireplaceQu"],:].fillna(value="None")
x_test.loc[x_test.isnull()["BsmtQual"],:] = x_test.loc[x_test.isnull()["BsmtQual"],:].fillna(value="None")
x_test.loc[x_test.isnull()["BsmtFinType1"],:] = x_test.loc[x_test.isnull()["BsmtFinType1"],:].fillna(value="None")
x_test.loc[x_test.isnull()["MasVnrType"],:] = x_test.loc[x_test.isnull()["MasVnrType"],:].fillna(value="None")

In [8]:
x_train.shape

(1460, 11)

In [9]:
x_test.shape

(1459, 11)

In [10]:
add_x = pd.concat([x_train,x_test])

In [11]:
one_hot = pd.get_dummies(add_x)

In [12]:
train_onehot=one_hot.iloc[0:1460,:]

In [13]:
test_onehot=one_hot.iloc[1460:2920,:]

---

In [14]:
from sklearn.preprocessing import MinMaxScaler 

In [30]:
x_data =df_train.loc[:,["OverallQual","YearBuilt","YearRemodAdd","MasVnrArea","TotalBsmtSF","1stFlrSF","GrLivArea","FullBath","TotRmsAbvGrd","Fireplaces","GarageCars","GarageArea"]]

In [31]:
x_data.isnull().sum().sort_values(ascending=False)

MasVnrArea      8
GarageArea      0
GarageCars      0
Fireplaces      0
TotRmsAbvGrd    0
FullBath        0
GrLivArea       0
1stFlrSF        0
TotalBsmtSF     0
YearRemodAdd    0
YearBuilt       0
OverallQual     0
dtype: int64

In [32]:
x_data.loc[x_data.isnull()["MasVnrArea"],:] = x_data.loc[x_data.isnull()["MasVnrArea"],:].fillna(value=x_data["MasVnrArea"].mean())

In [33]:
x_data_train = pd.concat([train_onehot,x_data], axis=1)

In [34]:
scaler = MinMaxScaler()

In [35]:
scaler.fit(x_data_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [36]:
x_data =scaler.transform(x_data_train)

In [39]:
y_data = df_train.iloc[:,-1]

In [40]:
y_data = y_data.values
y_data=y_data.reshape(-1,1)

In [41]:
print(x_data.shape)
print(y_data.shape)

(1460, 70)
(1460, 1)


In [46]:
X = tf.placeholder(tf.float32, shape=[None,70])
Y = tf.placeholder(tf.float32, shape=[None,1])


W = tf.Variable(tf.random_normal([70,1],name='weight'))
b = tf.Variable(tf.random_normal([1],name='bias'))

hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis-Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1)
train = optimizer.minimize(cost)

In [47]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(200001):
    cost_val, hy_val, _ = sess.run(
        [cost,hypothesis,train],
        feed_dict={X:x_data, Y:y_data})
    if step % 2000 == 0:
        print(step,"Cost : ",cost_val,
             "\nPreadiction\n",hy_val)

0 Cost :  39040250000.0 
Preadiction
 [[-3.5362575]
 [-2.1530204]
 [-5.7721195]
 ...
 [-1.3176692]
 [-4.665642 ]
 [-2.7892427]]
2000 Cost :  1084302300.0 
Preadiction
 [[209351.52 ]
 [177525.47 ]
 [219648.5  ]
 ...
 [250160.4  ]
 [130359.734]
 [139995.42 ]]
4000 Cost :  1070733400.0 
Preadiction
 [[208199.45]
 [177024.52]
 [218999.2 ]
 ...
 [249172.7 ]
 [131028.31]
 [140282.23]]
6000 Cost :  1062552060.0 
Preadiction
 [[207749.75]
 [176715.56]
 [219091.7 ]
 ...
 [250059.52]
 [131398.8 ]
 [140370.44]]
8000 Cost :  1056996900.0 
Preadiction
 [[207491.45]
 [176491.1 ]
 [219237.05]
 ...
 [250945.05]
 [131686.3 ]
 [140420.03]]
10000 Cost :  1053120060.0 
Preadiction
 [[207312.86]
 [176333.4 ]
 [219367.  ]
 ...
 [251662.53]
 [131933.84]
 [140453.47]]
12000 Cost :  1050364160.0 
Preadiction
 [[207176.95]
 [176226.3 ]
 [219475.81]
 ...
 [252248.7 ]
 [132154.6 ]
 [140480.19]]
14000 Cost :  1048376300.0 
Preadiction
 [[207067.83]
 [176156.23]
 [219565.75]
 ...
 [252738.14]
 [132353.53]
 [140503.

---

In [48]:
x_data =df_test.loc[:,["OverallQual","YearBuilt","YearRemodAdd","MasVnrArea","TotalBsmtSF","1stFlrSF","GrLivArea","FullBath","TotRmsAbvGrd","Fireplaces","GarageCars","GarageArea"]]

In [49]:
x_data.loc[x_data.isnull()["MasVnrArea"],:] = x_data.loc[x_data.isnull()["MasVnrArea"],:].fillna(value=x_data["MasVnrArea"].mean())
x_data.loc[x_data.isnull()["GarageCars"],:] = x_data.loc[x_data.isnull()["GarageCars"],:].fillna(value=x_data["GarageCars"].mean())
x_data.loc[x_data.isnull()["GarageArea"],:] = x_data.loc[x_data.isnull()["GarageArea"],:].fillna(value=x_data["GarageArea"].mean())
x_data.loc[x_data.isnull()["TotalBsmtSF"],:] = x_data.loc[x_data.isnull()["TotalBsmtSF"],:].fillna(value=x_data["TotalBsmtSF"].mean())

In [50]:
x_data.isnull().sum().sort_values(ascending=False)

GarageArea      0
GarageCars      0
Fireplaces      0
TotRmsAbvGrd    0
FullBath        0
GrLivArea       0
1stFlrSF        0
TotalBsmtSF     0
MasVnrArea      0
YearRemodAdd    0
YearBuilt       0
OverallQual     0
dtype: int64

In [51]:
x_data_test = pd.concat([test_onehot,x_data], axis=1)

In [52]:
x_data_test =scaler.transform(x_data_test)

In [53]:
x_data_test.shape

(1459, 70)

In [73]:
#x_data_test = MinMaxScaler().fit_transform(x_data_test)

---

In [54]:
sess.run(hypothesis, feed_dict={X:x_data_test})

array([[106809.42],
       [146351.08],
       [174512.95],
       ...,
       [160068.48],
       [110816.23],
       [216501.89]], dtype=float32)

In [55]:
sample = pd.read_csv("sample_submission.csv")

In [56]:
sample['SalePrice']=sess.run(hypothesis, feed_dict={X:x_data_test})

In [57]:
sample.to_csv("sample8.csv",index=False)

In [58]:
def log_rmse(predict_y, y_val):
    if predict_y.shape[0] != y_val.shape[0]:
        error_msg = "not matched length of arrays"
        return error_msg
    else:
        RMSE_log = np.sqrt(((np.log(predict_y) - np.log(y_val))**2).mean())
        return RMSE_log

In [59]:
y_train =df_train.iloc[:,-1]

In [60]:
log_rmse(hy_val.reshape(-1,), y_train)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log
  


0.155009267656005